In [ ]:
# import necessary modules
import pandas as pd
import os

In [ ]:
# define the source data paths
students_csvPath = os.path.join("Resources", "students_complete.csv")
schools_csvPath = os.path.join("Resources", "schools_complete.csv")

In [ ]:
# read the data into their initial dataframs
students_df = pd.read_csv(students_csvPath)
schools_df = pd.read_csv(schools_csvPath)

In [ ]:
# define threshold values
passingThreshold = 70

In [ ]:
# calculate the district summary metrics
totalSchools = len(schools_df["School ID"])
totalStudents = len(students_df["student_name"])
totalBudget = schools_df["budget"].sum()
avgMathScore = students_df["math_score"].mean()
avgReadingScore = students_df["reading_score"].mean()
passingMath = sum(students_df.math_score >= passingThreshold) / totalStudents * 100
passingReading = sum(students_df.reading_score >= passingThreshold) / totalStudents * 100
passingBoth = sum(
    (students_df.math_score >= passingThreshold) & 
    (students_df.reading_score >= passingThreshold)
    ) / totalStudents * 100

In [ ]:
# calculate the school summary metrics


In [ ]:
# create the district summary DataFrame
district_df = pd.DataFrame({
    "Total Schools": [totalSchools],
    "Total Students": f"{totalStudents:,.0f}",
    "Total Budget": f"${totalBudget:,.2f}",
    "Avg Math Score": f"{avgMathScore:,.2f}",
    "Avg Reading Score": f"{avgReadingScore:,.2f}",
    "Passing Math": f"{passingMath:,.2f}%",
    "Passing Reading": f"{passingReading:,.2f}%",
    "Passing": f"{passingBoth:,.2f}%"})

In [ ]:
# create the school summary DataFrame
school_df = pd.DataFrame({
    
})